In [1]:
import torch
from torchvision.ops import nms
from torch.utils.data import DataLoader
from yolox.test_weights import download_weights, load_pretrained_weights
from yolox.model import create_yolox_s
from data_utils.ppe_dataset import PPE_DATA

In [2]:
num_classes = 80
weight_path = download_weights("yolox/yolox_s.pth")
model = create_yolox_s(num_classes)
model = load_pretrained_weights(model, weight_path, num_classes)

Loading weights from yolox/yolox_s.pth
Loaded weights:
  Missing keys: 0
  Unexpected keys: 0


In [4]:
model.eval()
dataset = PPE_DATA()

def yolo_collate(batch):
    imgs   = torch.stack([b[0] for b in batch])   #  (B,3,640,640)
    labels = [b[1] for b in batch]                #  list of length B
    return imgs, labels

loader = DataLoader(dataset, batch_size=16, shuffle = False, collate_fn=yolo_collate)

In [ ]:
for batch in loader:
    img, label = batch
    img = img.cuda()
    model = model.cuda()
    
    with torch.no_grad():
        # Forward pass through the model
        # noise = torch.randn_like(img).cuda()
        output = model(img)
        obj = output[..., 4:5]
        print(obj.max())

tensor(0.9902, device='cuda:0')
tensor(0.9915, device='cuda:0')
tensor(0.9921, device='cuda:0')
tensor(0.9904, device='cuda:0')
tensor(0.9820, device='cuda:0')
tensor(0.9880, device='cuda:0')


KeyboardInterrupt: 

In [9]:
x1 = output[..., 0:1] - output[..., 2:3] / 2
y1 = output[..., 1:2] - output[..., 3:4] / 2
x2 = output[..., 0:1] + output[..., 2:3] / 2
y2 = output[..., 1:2] + output[..., 3:4] / 2
boxes = torch.cat([x1, y1, x2, y2], dim=-1)

obj = output[..., 4:5]
class_probs = output[..., 5:]
scores = obj * class_probs
print(torch.max(class_probs))
best_scores, best_class = scores.max(dim=-1)

mask = best_scores > 0.25
best_scores, best_class, boxes = best_scores[mask], best_class[mask], boxes[mask]
# TODO what abt batches??
boxes = boxes.squeeze(0)
best_scores = best_scores.squeeze(0)
print(best_scores.shape, boxes.shape, best_class.shape)
keep = nms(boxes, best_scores, iou_threshold=0.5)
final_boxes = boxes[keep]
final_classes = best_class.squeeze(0)[keep]

tensor(0.5890, device='cuda:0')
torch.Size([0]) torch.Size([0, 4]) torch.Size([0])


In [12]:
labels = torch.cat((final_classes.unsqueeze(1), final_boxes), dim=1)
print(labels.shape)
if labels.shape[0] == 0:
    print("No detections found.")
else:
    PPE_DATA.show_img(img, labels[:20, :], rect_coords_centered=False)

torch.Size([2630, 5])
tensor(0., grad_fn=<UnbindBackward0>)
tensor(0., grad_fn=<UnbindBackward0>)
tensor(0., grad_fn=<UnbindBackward0>)
tensor(0., grad_fn=<UnbindBackward0>)
tensor(0., grad_fn=<UnbindBackward0>)
tensor(0., grad_fn=<UnbindBackward0>)
tensor(0., grad_fn=<UnbindBackward0>)
tensor(0., grad_fn=<UnbindBackward0>)
tensor(0., grad_fn=<UnbindBackward0>)
tensor(0., grad_fn=<UnbindBackward0>)
tensor(0., grad_fn=<UnbindBackward0>)
tensor(0., grad_fn=<UnbindBackward0>)
tensor(0., grad_fn=<UnbindBackward0>)
tensor(0., grad_fn=<UnbindBackward0>)
tensor(0., grad_fn=<UnbindBackward0>)
tensor(0., grad_fn=<UnbindBackward0>)
tensor(15., grad_fn=<UnbindBackward0>)
tensor(0., grad_fn=<UnbindBackward0>)
tensor(0., grad_fn=<UnbindBackward0>)
tensor(0., grad_fn=<UnbindBackward0>)
